<a href="https://colab.research.google.com/github/bell-avery/fig2/blob/master/Figure_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import glob, os
import numpy as np
import seaborn as sns

In [0]:
up_regress = pd.DataFrame({})

for f in os.listdir('BonCorrect'):
  if f.find('up') > 0:
    this_tsv = pd.read_csv('BonCorrect/' + f, delimiter='\t', encoding='utf-8')
    cancer_type = f.split('_')[0]
    this_tsv['cancer_type'] = [cancer_type for _ in range(this_tsv.shape[0])]
    up_regress = pd.concat([up_regress, this_tsv])

up_regress.head()

In [0]:
up_cnv = {}

for f in os.listdir('CNV Tables'):
  if f.find('.txt') > 0 and f.find('up') > 0:
    this_tsv = pd.read_csv('CNV Tables/' + f, delimiter='\t', encoding='utf-8', index_col=0)
    cancer_type = f.split('_')[0]
    up_cnv[cancer_type] = this_tsv

print(up_cnv)

In [0]:
up_protein = {}

for f in os.listdir('Protein tables'):
  this_tsv = pd.read_csv('Protein tables/' + f, delimiter='\t', encoding='utf-8')
  cancer_type = f.split('ProData')[0]
  if 'Patient_ID' in this_tsv.columns:
    this_tsv = this_tsv.set_index('Patient_ID')
  else:
    this_tsv = this_tsv.set_index('Name')
  up_protein[cancer_type] = this_tsv

print(up_protein)

In [0]:
regress_pivot = up_regress[['CNV', 'Protien', 'slope', 'cancer_type']]
regress_pivot['gene_prot_pair'] = regress_pivot[['CNV', 'Protien']].apply(lambda x: '/'.join(x.dropna().astype(str).values), axis=1)
regress_pivot = regress_pivot.pivot(index='cancer_type',columns='gene_prot_pair', values='slope')

sig_regress_pivot = regress_pivot.loc[:, regress_pivot.isna().sum() < 2]
sig_regress_pivot = sig_regress_pivot.dropna()

In [0]:
sns.clustermap(sig_regress_pivot.loc[:,[p.find("NUDCD1") >= 0 for p in sig_regress_pivot.columns]], figsize=(15, 8), cbar_pos=(0, 0, .05, .3))
plt.title("slope") 
plt.savefig('Figures/NUDCD1_heatmap.jpg')

In [0]:
sns.clustermap(sig_regress_pivot.loc[:,[p.find("YWHAZ") >= 0 for p in sig_regress_pivot.columns]], figsize=(15, 8), cbar_pos=None)
plt.savefig('Figures/YWHAZ_heatmap.jpg')